<a href="https://colab.research.google.com/github/Michael-Bokov/Data_Science/blob/main/NLP(SGD_SVC)IMDB_comments_classification(classic_methods).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install 'portalocker>=2.0.0'

In [2]:
pip install torch==1.13.0

In [3]:
!pip install scipy==1.7.3 scikit-learn==1.0.2

In [4]:
pip install torchtext==0.14.0 torchdata==0.5.0

In [5]:
from torchtext.datasets import IMDB

In [15]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline

In [22]:
from sklearn.metrics import classification_report, confusion_matrix

In [6]:
# создадим итератор по данным, загрузив готовый датасет IMDB из torchtext,
# причем для начала, только обучающую train подвыборку, используя аргумент split
# нам нужно понять, какие есть классы в датасете, поэтому проитерируемся по всей
# обучающей подвыборке и соберем все представленные метки классов
torch_train=iter(IMDB(split='train'))
texts=[]
labels=[]
for label,text in torch_train:
  labels += [label]
  texts += [text]
# определим список классов в датасете, взяв только уникальные метки,
# превратив список классов в set и затем обратно в list, и
# отсортировав их "для красоты" с помощью функции sorted
CLASSES = sorted(list(set(labels)))
print(CLASSES)
#print(texts[:3])
#несколько примеров из датасета.
n_samples,i=3,0
while i<n_samples:
  print(f"'text:'{texts[i]} ,'label:'{labels[i]}")
  #print(f'label:'{labels[i]})
  i+=1


[1, 2]
'text:'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and fa

In [7]:
# создадим DataFrame, передав ему словарь,
# в котором будет два ключа: text и label, а в значениях листы примеров texts и labels
import pandas as pd
data_train=pd.DataFrame({'text':texts,'label':labels})
data_train.head(5)

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,1
1,"""I Am Curious: Yellow"" is a risible and preten...",1
2,If only to avoid making this type of film in t...,1
3,This film was probably inspired by Godard's Ma...,1
4,"Oh, brother...after hearing about this ridicul...",1


Если бы в нашем датасете лейблы были заданы в виде строк-названий, было бы необходимо перевести их в числовой формат, для этого можно воспользоваться LabelEncoder. В нашем случае метки классов уже представлены в виде 1 и 2, поэтому просто будут переведены в 0 и 1.

from sklearn.preprocessing import LabelEncoder
создадим инстанс LabelEncoder с дефолтными параметрами
le = LabelEncoder()
обучим наш le, используя метод fit, на всех метках датасета df["label"]
le.fit(df["label"])
присвоим в новый столбец encoded_label датафрейма df обработанные метки классов.
метки классов df["label"] обрабатываются с помощью метода transform,
применяемого к le
df['encoded_label'] = le.transform(df["label"])
распечатаем полученные уникальные значения в столбце encoded_label
c помощью метода unique()

In [8]:
data_train['label'].replace({1:0,2:1}, inplace=True)
data_train.rename(columns={"label": 'encoded_label'}, inplace=True)
data_train['encoded_label'].unique()

array([0, 1])

In [9]:
#Разделим датасет на обучающую и тестовую выборки (20%)  передадим также наши метки классов в качестве параметра stratify
# для сбалансированного разделения на обучающую и тестовую выборки,
# зафиксируем random_state
from sklearn.model_selection import train_test_split
df=data_train.copy()
x_train,x_test,y_train,y_test = train_test_split(
    df['text'], df['encoded_label'], test_size=0.2,
    stratify=df['encoded_label'], random_state=42)

In [55]:
x_train.head(5)

20022    I have always been a huge James Bond fanatic! ...
4993     I am a Christian and I say this movie had terr...
24760    Neatly sandwiched between THE STRANGER, a smal...
13775    Years ago I did follow a soap on TV. So I was ...
20504    Here's a gritty, get-the-bad guys revenge stor...
Name: text, dtype: object

# SGD  Stochastic Gradient Decent  Стохастический градиентный спуск


# "Ручная if-idf векторизация"

Произведем TF-IDF векторизацию текста.
Создадим инстанс TfidfVectorizer, используя следующие параметры:
используем униграммы, задав значение (1,1)(отдельные слова) для аргумента ngram_range,
текст будем делить на слова, задав значение 'word' аргумента analyzer,
зададим значение min_df равное 2, чтобы в словарь добавлялись все ngrams, встретившиеся больше 2 раз,
зададим стоп-слова "english" с помощью аргумента stop_words.
То есть создадим словарь слов встречающихся более 2 раз.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,1),
                             analyzer='word',
                             min_df=2,
                             stop_words="english")
# обучаем словарь и сразу преобразуем обучающую выборку x_train с помощью метода fit_transform
tfidf_train = vectorizer.fit_transform(x_train)
# преобразуем тестовую выборку x_test с помощью метода transform
tfidf_test = vectorizer.transform(x_test)

In [46]:
# Размер словаря
print(tfidf_train.shape[1])

40525


In [31]:
# Что представляет собой один пример данных после TFIDF
tfidf_train[0].data

array([0.07019251, 0.07937233, 0.07525749, 0.09475886, 0.10671608,
       0.10671608, 0.10358497, 0.10671608, 0.09475886, 0.10115629,
       0.09917191, 0.09475886, 0.07340832, 0.18325551, 0.10358497,
       0.09917191, 0.07676857, 0.08553694, 0.05685072, 0.10671608,
       0.09917191, 0.10671608, 0.10358497, 0.10115629, 0.17213593,
       0.09749415, 0.0872147 , 0.09361213, 0.10671608, 0.10358497,
       0.10671608, 0.06197363, 0.07139564, 0.0960408 , 0.10358497,
       0.10671608, 0.0960408 , 0.09075657, 0.08553694, 0.06877085,
       0.10671608, 0.07111655, 0.10671608, 0.08606796, 0.10671608,
       0.10671608, 0.10671608, 0.10115629, 0.09917191, 0.10115629,
       0.07486166, 0.06922441, 0.10115629, 0.10671608, 0.09361213,
       0.07466902, 0.09917191, 0.07879936, 0.07525749, 0.10671608,
       0.15721615, 0.10671608, 0.10671608, 0.10671608, 0.20716993,
       0.10671608, 0.10671608, 0.08202389, 0.0960408 , 0.10671608,
       0.10671608, 0.10671608, 0.14647914, 0.31748932, 0.10358

In [ ]:
#Векторизованные тексты хранятся в виде разреженной матрицы.
tfidf_train

Для обучения векторные представления необходимо преобразовать в плотную матрицу. Реализуем  с помощью метода scipy.sparse.csr_matrix.todense

In [11]:
import scipy

train_tfidf_dense = scipy.sparse.csr_matrix.todense(tfidf_train)#[:3, :])
print(train_tfidf_dense[0]) # [train_tfidf_dense[0] != 0][0]
print(len(train_tfidf_dense))
print(train_tfidf_dense[0][train_tfidf_dense[0] != 0][0])

[[0. 0. 0. ... 0. 0. 0.]]
20000
[[0.15294496 0.09187453 0.06404302 0.03773019 0.04907279 0.08645197
  0.07330856 0.05847762 0.05570911 0.09219234 0.02388042 0.02451574
  0.30800698 0.10994039 0.03797819 0.38215483 0.05970551 0.07444551
  0.0240369  0.22333652 0.03490929 0.05013343 0.13961062 0.03094719
  0.07010147 0.07229822 0.02421292 0.04161227 0.04776302 0.02458804
  0.06624411 0.13755173 0.05402733 0.03703425 0.04520614 0.02845448
  0.03368561 0.06531163 0.03749915 0.0244194  0.0709112  0.04822389
  0.02789725 0.07096651 0.08242689 0.05128706 0.15817141 0.03548581
  0.02829875 0.08645197 0.05182299 0.06523855 0.05228628 0.05223362
  0.08382133 0.03921774 0.0678513  0.07970681 0.12054531 0.06058324
  0.05797002 0.03229802 0.06817942 0.05563246 0.01698921 0.17290393
  0.13691421 0.02410548 0.03201017 0.0443074  0.05923957 0.02862854
  0.02465335 0.05073637 0.02190686 0.0258912  0.03795636 0.06949401
  0.04273943 0.05005048 0.05244611 0.01353336 0.04603478 0.19253527
  0.0573039  0.0

In [19]:
test_tfidf_dense = scipy.sparse.csr_matrix.todense(tfidf_test)

Создаем пайплайн с классификатором.

In [16]:
pipeline_1 = Pipeline([
        ("classifier", SGDClassifier(
        loss="log",
        penalty="l2",
        class_weight="balanced",
        tol=0.001))])

In [17]:
pipeline_1.fit(train_tfidf_dense, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Pipeline(steps=[('classifier',
                 SGDClassifier(class_weight='balanced', loss='log'))])

In [20]:
y_test_pred = pipeline_1.predict(test_tfidf_dense)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


# Результаты

In [23]:
# распечатаем метрики с помощью функции classification_report,
# в которую в качестве аргументов передаются y_test и y_test_pred
print(classification_report(y_test, y_test_pred))
# распечатаем confusion matrix с помощью функции confusion_matrix,
# в которую в качестве аргументов передаются y_test и y_test_pred
print(f"Confusion matrix:\n{confusion_matrix(y_test, y_test_pred)}")

              precision    recall  f1-score   support

           0       0.90      0.87      0.88      2500
           1       0.87      0.90      0.89      2500

    accuracy                           0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000

Confusion matrix:
[[2166  334]
 [ 247 2253]]


# "Векторизация из коробки"



Строим и обучим классификатор.
Две модели: Stochastic Gradient Decent и Support Vector Machine. (СГД и метод опорных векторов). Cоздадим инстанс Pipeline, передав в качестве аргумента лист из двух моделей(tuple из двух элементов: строка-название и сам инстанс модели)


In [24]:
pipeline = Pipeline([
    ("feature_extractor", TfidfVectorizer(
        input="array",
        norm="l2",
        max_features=None,
        sublinear_tf=True,
        stop_words="english")),
    ("classifier", SGDClassifier(
        loss="log",
        penalty="l2",
        class_weight="balanced",
        tol=0.001))])

In [25]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('feature_extractor',
                 TfidfVectorizer(input='array', stop_words='english',
                                 sublinear_tf=True)),
                ('classifier',
                 SGDClassifier(class_weight='balanced', loss='log'))])

In [26]:
y_test_pred = pipeline.predict(x_test)

In [27]:
y_test_pred[:10]

array([1, 0, 0, 0, 1, 0, 1, 0, 0, 0])

In [28]:
print(classification_report(y_test, y_test_pred))
print(f"Confusion matrix:\n{confusion_matrix(y_test, y_test_pred)}")

              precision    recall  f1-score   support

           0       0.90      0.86      0.88      2500
           1       0.87      0.91      0.89      2500

    accuracy                           0.88      5000
   macro avg       0.89      0.88      0.88      5000
weighted avg       0.89      0.88      0.88      5000

Confusion matrix:
[[2150  350]
 [ 229 2271]]


#SVC  Support Vector Machine. Метод опорных векторов

In [29]:
from sklearn.svm import SVC
pipeline_2 = Pipeline([
    ("feature_extractor", TfidfVectorizer(
        input="array",
        norm="l2",
        # для ускорения обучения ограничим максимальное число признаков
        max_features=1000,
        sublinear_tf=True,
        stop_words="english")),
    ("classifier", SVC(
        C=10,
        kernel="rbf",
        gamma=0.1,
        probability=True,
        class_weight=None)),
        ]
)

In [31]:
pipeline_2.fit(x_train, y_train)

y_test_pred = pipeline.predict(x_test)


#Результаты

In [32]:

print(classification_report(y_test, y_test_pred))

print(f"Confusion matrix:\n{confusion_matrix(y_test, y_test_pred)}")

              precision    recall  f1-score   support

           0       0.90      0.86      0.88      2500
           1       0.87      0.91      0.89      2500

    accuracy                           0.88      5000
   macro avg       0.89      0.88      0.88      5000
weighted avg       0.89      0.88      0.88      5000

Confusion matrix:
[[2150  350]
 [ 229 2271]]
